# Actualiza horarios de clases desde grace

## Importes y utilidades

In [1]:
import requests
import urllib.request
import json
import os
from bs4 import BeautifulSoup

In [2]:
def getHTML(url):
    with urllib.request.urlopen(url) as u:
        html=u.read().decode("utf-8","ignore")
    return html

In [3]:
#TODO creo que esto no es levenshtein xd
def levenshtein_ratio(a,b):
    a=a.lower().strip()
    b=b.lower().strip()
    edits_away=0
    for index,letter in enumerate(a):
        try:
            if letter!=b[index]:edits_away+=1
        except:edits_away+=1
    ratio=((len(a)+len(b)) - edits_away) / (len(a)+len(b))
    return ratio

In [4]:
def replace_latin_chars(str):
    translate = {"Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U", "Ñ": "N", "Ü": "U","á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u", "ñ": "n", "ü": "u"}
    for original,replacement in translate.items():
        str.replace(original,replacement)
    return str

In [5]:
def fix_url(str):
    nombre = str.strip();
    nombre = replace_latin_chars(nombre);
    nombre = nombre.replace("/[^a-zA-Z0-9\s]/", "");
    nombre = nombre.replace("(", "");
    nombre = nombre.replace(")", "");
    nombre = nombre.replace(" -", "-");
    nombre = nombre.replace("- ", "-");
    nombre = nombre.replace("\n", "");
    nombre = nombre.replace("\r", "");
    nombre = nombre.replace("  ", " ");
    nombre = nombre.replace("\\", "");
    nombre = nombre.replace(" ", "-");
    return nombre;

## Scrap grace

In [6]:
#Regresa periodo y lista de nombre de clases dado el url del dropdown de grace
def preliminaryInfo(url):
    html=getHTML(url)
    periodo=(html.split("período")[1].split("</h3>")[0]).strip()
    clases=[i.replace('</option>','').replace('\n','') for i in html.split("<option>")[1:-1]]  
    return periodo,clases

In [7]:
#Dado el html de form_url regresa diccionario de la forma {"grupo":{diccionario con info de grupo}} y un booleado laboratorio
def getClaseInfo(html):
    if not html:return None
    grupos={}
    teoria=False
    laboratorio=False
    info=["depto","clave","grupo","TL","nombre","prof","cred","horario","dias","salon","campus","comentarios"] #datos por extraer
    soup=BeautifulSoup(html, 'html.parser')
    tabla=soup.find_all('table')[2] #la tabla con la info es la 3ra en el html
    for renglon in tabla.find_all('tr')[1:]: #saltar renglon con cabecera
        grupo={i:None for i in info}
        info_index=0 #indice para navegar info[]
        for columna in renglon.find_all('td'):
            grupo[info[info_index]]=columna.string.strip()
            if info[info_index]=="dias":
                grupo[info[info_index]]=grupo[info[info_index]].split()
            info_index+=1
        if grupo["TL"]=="L":
            grupo["grupo"]+="L"
        #Arreglar horario en inicio y fin      
        inicio,fin=grupo["horario"].split("-")
        grupo["inicio"]=inicio
        grupo["fin"]=fin
        grupos[grupo["grupo"]]=grupo
        
    return grupos

In [8]:
#Returns dict w clave,nombre,grupos
def formatClases(clases):
    formated={} #clave: info grupo
    for c in clases:
        grupos=[]
        gruposLab=[]
        for clave,data in c.items():
            originalNombre=data['depto']+'-'+data['clave']+'-'+data['nombre']
            nombre=originalNombre if data['TL']=='T' else originalNombre+'-LAB'
            d={'grupo':data['grupo'],'nombre':nombre,'profesor':data['prof'],'creditos':data['cred'],'horario':data['horario'],'dias':data['dias'],'salon':data['salon'],'campus':data['campus'],'inicio':data['inicio'],'fin':data['fin']}
            if data['TL']=='L':
                gruposLab.append(d)
            elif data['TL']=='T':
                grupos.append(d)
        if len(gruposLab)>0:
            #print(len(grupos),len(gruposLab))
            l={'nombre':originalNombre+'-LAB','clave':data['depto']+'-'+data['clave'],'grupos':gruposLab}
            formated[l['clave']+'-LAB']=l
        t={'nombre':originalNombre,'clave':data['depto']+'-'+data['clave'],'grupos':grupos}
        formated[t['clave']]=t
    return formated

In [9]:
s="1995"
dropDownUrl="https://serviciosweb.itam.mx/EDSUP/BWZKSENP.P_Horarios1?s="+s
formPostUrl="https://serviciosweb.itam.mx/EDSUP/BWZKSENP.P_Horarios2"

In [10]:
periodo,listaClases=preliminaryInfo(dropDownUrl)
periodo,len(listaClases)

('PRIMAVERA 2022 LICENCIATURA', 425)

In [11]:
clases=[getClaseInfo(requests.post(url=formPostUrl,data={"s":s,"txt_materia":nombre}).text) for nombre in listaClases]

In [12]:
clases

[{'001': {'depto': 'ACT',
   'clave': '11300',
   'grupo': '001',
   'TL': 'T',
   'nombre': 'CALCULO ACTUARIAL I',
   'prof': 'SERGIO GARCIA ALQUICIRA',
   'cred': '6',
   'horario': '19:00-20:30',
   'dias': ['MA', 'JU'],
   'salon': 'RH108',
   'campus': 'RIO HONDO',
   'comentarios': '',
   'inicio': '19:00',
   'fin': '20:30'},
  '002': {'depto': 'ACT',
   'clave': '11300',
   'grupo': '002',
   'TL': 'T',
   'nombre': 'CALCULO ACTUARIAL I',
   'prof': 'PEDRO DANIEL TREJO SOLIS',
   'cred': '6',
   'horario': '19:00-20:30',
   'dias': ['LU', 'MI'],
   'salon': 'RH311',
   'campus': 'RIO HONDO',
   'comentarios': '',
   'inicio': '19:00',
   'fin': '20:30'},
  '003': {'depto': 'ACT',
   'clave': '11300',
   'grupo': '003',
   'TL': 'T',
   'nombre': 'CALCULO ACTUARIAL I',
   'prof': 'FERNANDO PEREZ RIOS',
   'cred': '6',
   'horario': '07:00-08:30',
   'dias': ['LU', 'MI'],
   'salon': 'RHPB3',
   'campus': 'RIO HONDO',
   'comentarios': '',
   'inicio': '07:00',
   'fin': '08:30'}

In [13]:
formatedClases=formatClases(clases)

In [14]:
with open("data.js","w+") as f:
    f.write("let periodo='"+periodo+"';")
    f.write("\nlet sGrace='"+s+"';")
    f.write("\nlet dropDownUrl='"+dropDownUrl+"';")
    f.write("\nlet formPostUrl='"+formPostUrl+"';")
    f.write("\nlet clases="+json.dumps(formatedClases,indent=2)+";")

## Scrap MisProfes.com

In [15]:
misProfesUrl='https://www.misprofesores.com/escuelas/ITAM-Instituto-Tecnologico-Autonomo-de-Mexico_1003'
misProfesHTML=getHTML(misProfesUrl)
def profUrl(nombre,apellido,id):
    return "https://www.misprofesores.com/profesores/"+fix_url(nombre)+"-"+fix_url(apellido)+"_"+id;

In [16]:
start=misProfesHTML.index('var dataSet = ')+len('var dataSet = ')
tempData=json.loads(misProfesHTML[start:misProfesHTML.index('];',start)+1])
misProfesData={p['n']+' '+p['a']:{'c':p['c'],'m':p['m'],'i':p['i'],'misProfesNom':p['n']+' '+p['a'],'link':profUrl(p['n'],p['a'],p['i'])} for p in tempData}

## Match profesores

In [17]:
profesores=list(set([grupo['prof'] for clase in clases for grupo in clase.values() if len(grupo['prof'].strip())>1]))

In [18]:
matchedData={p:misProfesData[m] for m in misProfesData.keys() for p in profesores if levenshtein_ratio(p,m)>0.90}

In [19]:
matchedData

{'CLAUDIA ABURTO RANCAÑO': {'c': '8.9324',
  'm': '74',
  'i': '10763',
  'misProfesNom': 'Claudia Aburto Rancaño',
  'link': 'https://www.misprofesores.com/profesores/Claudia-Aburto-Rancaño_10763'},
 'MARIA DE LAS MERCEDES ADAMUZ PEÑA': {'c': '6.5833',
  'm': '24',
  'i': '10746',
  'misProfesNom': 'María de las Mercedes Adamuz Peña',
  'link': 'https://www.misprofesores.com/profesores/María-de-las-Mercedes-Adamuz-Peña_10746'},
 'ARTURO ALBERTO AGUILAR ESTEVA': {'c': '8.5385',
  'm': '13',
  'i': '58325',
  'misProfesNom': 'Arturo Alberto Aguilar Esteva',
  'link': 'https://www.misprofesores.com/profesores/Arturo-Alberto-Aguilar-Esteva_58325'},
 'JOSE ABRAHAM AGUILAR ICAZA': {'c': '7.0625',
  'm': '16',
  'i': '62617',
  'misProfesNom': 'Jose Abraham Aguilar Icaza',
  'link': 'https://www.misprofesores.com/profesores/Jose-Abraham-Aguilar-Icaza_62617'},
 'JUAN CARLOS AGUILAR VILLEGAS': {'c': '6.8654',
  'm': '52',
  'i': '10711',
  'misProfesNom': 'Juan Carlos Aguilar Villegas',
  'lin

## Write mis profes data

In [20]:
links={p:d['link'] for p,d in matchedData.items()}
ratings={p:[float(d['c']),float(d['m'])] for p,d in matchedData.items() if d['c']}

In [21]:
len(links)

312

In [22]:
len(ratings)

303

In [23]:
misProfesData={
    prof:{
        "link":links[prof],
        "general":ratings[prof][0],
        "n":int(ratings[prof][1])
    }
    for prof in ratings.keys()
}

In [24]:
with open("data.js","a") as f:
    f.write("\nlet misProfesData="+json.dumps(misProfesData,indent=2)+";")    